In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain

from dotenv import load_dotenv
load_dotenv()

True

# Summarization Node
  - 모든 프롬프트는 yaml 파일로 저장하고, langchain hub에 저장
  - Document의 크기에 따라 적합 요약 방법을 적용할 수 있는 Node
  - Document의 content 크기가 2개 이상인 경우, map-reduce, map-refine 방식을 적용하여 평가함

### 요약 기법 
1. Stuff: 전체 문서 한번에 요약
2. Map-Reduce: 분할 요약 후 일괄 병합
3. Map-Refine: 분할 요약 후 점진적인 병합

# Summarization 평가
  - heading을 기준으로 parsing을 진행하기 때문에, document내 element 갯수의 편차가 존재
  - element의 갯수와 글자 수를 기준으로 적절한 요약 기법을 선택함으로써 비용의 효율화 및 향상된 요약 품질을 유도

### 평가지표
1. 명확성 (Clarity)
  - 요약문이 명확하고 이해하기 쉬운가?
  - 모호한 표현이나 불분명한 문장이 없는가?
  - 문장 구조가 논리적인가?

2. 간결성 (Conciseness)
  - 불필요한 반복이나 중복된 정보가 없는가?
  - 핵심 내용만 간단명료하게 전달하는가?
  - 문장이 경제적으로 구성되었는가?

3. 완전성 (Completeness)
  - 원문의 주요 내용이 모두 포함되었는가?
  - 중요한 정보가 누락되지 않았는가?
  - 핵심 아이디어가 모두 반영되었는가?

4. 일관성 (Consistency)
  - 요약이 원문의 맥락과 일치하는가?
  - 원문의 의도나 논조가 왜곡되지 않았는가?
  - 내용 간의 논리적 연결이 자연스러운가?

5. 유용성 (Usefulness)
  - 요약이 실제로 유용한 정보를 제공하는가?
  - 독자가 원문을 이해하는 데 도움이 되는가?
  - 실용적인 가치가 있는가?

In [2]:
import os
import json

with open('../data/document/역도/documentParseGraph_state.json', 'r') as f:
    state = json.load(f)

### 평가 DataFrame 생성

In [3]:
import pandas as pd

# result_df = pd.DataFrame(columns=['document_index', 'num_paragraph', 'num_letter','summarization_method', 'cost', 'response', 'summary', 'clarity', 'brevity', 'completeness', 'consistency', 'usefulness'])
# result_df.to_csv('../data/experiment/summarization_result.csv', index=False)

result_df = pd.read_csv('../data/experiment/summarization_result.csv')


In [4]:
result_df.head()

,document_index,num_paragraph,num_letter,summarization_method,cost,response,summary,clarity,brevity,completeness,consistency,usefulness


In [40]:
def preprocess_sentence(text):
    text = re.sub(r'[\n\r]+', ' ', text)
    text = re.sub(r' {2,}', ' ', text)
    
    return text

In [45]:
# map-reduce, refine, chain of density 방식 적용 대상 선정

experiment_docs = [doc for doc in state['documents'] if len([content for content in doc['content'] if content['category'] == 'paragraph']) > 1]

In [52]:
tmp = experiment_docs[2]
tmp_heading = tmp['meta']['heading']

tmp_heading = '\n'.join([heading for heading in tmp_heading.values() if heading != None])
tmp_content = preprocess_sentence(' '.join([content['text'] for content in tmp['content'] if content['category'] == 'paragraph']))
tmp_origin = tmp_heading + tmp_content

tmp_origin

'Ⅴ.역도 훈련프로그램 구성 및 지도안\n1. 훈련프로그램의 구성 원리\n나. 훈련량의 설정 및 훈련프로그램의 실례\n1) 총부하량(volume of load)총부하량이란 연습기간, 또는 한정 된 주기 즉, 주, 월, 년, 단위에서 들어 올린 총중량을 의미하며, 대개 킬로그램(kg)이나, 톤(ton)으로 표시한다. 예) 100kg의 중량을 가지고 Snatch를 3세트 3회 반복하면, 100×3×3=900kg 120kg의 중량을 가지고 Snatch Pull을 5세트 5회 반복하면, 120×5×5=3000kg 150kg의 중량을 가지고 Back Squat을 10세트 10회 반복하면, 150×10×10=15000kg 이때의 총부하량은 900+3000+15000=18900kg(18.9t) 이 된다는 것이다. 이 수치는 선수가 행한 총 운동량을 나타내는 것이며, 총부하량은 체계적인 훈련기간을 단위 로 하여 주 단위, 월 단위, 년 단위로 산출하는 것이 보통이며, 훈련계획을 수립하 기 위하여 자신의 최대 총부하량을 기준으로 하여 낮은 수준의 부하량(60% 이하 ), 중간 수준의 부하량(60-80%), 높은 수준의 부하량(80% 이상)을 설정한다. 총부하량은 선수의 운동경력이나, 체급, 체력에 따라 설정범위도 달라야 하며, 최 근연구에 의하면 총부하량을 급격히 증가하면, 오히려 경기력에 부정적인 효과를 초 래 할 수도 있으며, 이는 훈련량의 한계를 시사해 주는 것으로 무리한 운동량의 설 정은 운동 상해를 야기 시키며, 선수생활을 단축시키는 결과를 초래하기 때문이다. 특히, 중요한 것은 원상회복과 훈련의 효과는 훈련량의 증가뿐만 아니라 훈련이 나 시합후의 회복과도 밀접한 관계가 있음이 밝혀졌으며, 훈련의 초기 단계에서 미 처 회복되지 않은 상태에서 훈련효과에 대한 분석을 한 결과 감소현상이 나타났다 고 한다. 즉, 과도한 훈련량, 특히 과도한 중량을 가지고 무리한 훈련을 할 경우에 는 역학적, 동역학적 매개 변수에 대한 정확한 기술을 구사할 수 없게 되어 운동수 행능력의 향

In [53]:
tmp_summary = "총부하량은 주, 월, 년 등 일정 기간 동안 운동 중에 들어 올린 총중량을 의미하며, 예를 들어 Snatch, Snatch Pull, Back Squat와 같이 각 운동별 세트와 반복 수를 곱하여 계산된 중량의 합계로 산출된다. 선수들은 자신의 최대 총부하량을 기준으로 60% 이하의 저부하, 60~80%의 중간부하, 80% 이상의 고부하로 나누어 훈련 프로그램에 반영하는데, 이는 급격한 부하량 증가는 경기력 저하와 부상의 위험을 초래할 수 있기 때문이다. 또한, 훈련의 효과는 단순히 부하량 증가뿐만 아니라 훈련 후 회복과도 밀접하게 관련되어 있어, 큰 부하 후에는 반드시 회복을 위한 적절한 작은 부하를 배치해야 한다. 월간 총부하량은 elite 선수들의 경우 시합 1개월 전 보통 100-150톤 정도로 설정되지만, 일반 선수들이 200-300톤의 과도한 부하로 훈련하는 것은 바람직하지 않다. 주간 총부하량은 이미 정해진 총부하량을 고려하여 시합 4주 전부터 시합 당일까지 주별로 조절하며, 우수 선수들의 경우 주간 부하량을 첫날 54%, 다음날 30%, 셋째날 16%로 분배하는 사례가 있지만, 이는 초보자에게 그대로 적용하기에는 무리가 있다. 이러한 체계적인 부하량 설정과 회복 고려는 훈련 효과를 극대화하고 선수의 경기력 향상에 필수적인 요소로 작용한다."

## 문장 전처리
* stuff 방식과 map-reduce, refine 방식의 전처리 방식이 다름
  * struff 방식의 경우, 불용어 제거하고 element의 text를 하나로 합침
  * map-reduce, refine 방식의 경우, 불용어 제거하고 element를 분리

In [32]:
# stuff 방식
import re

def stuff_preprocess_document(document):
    num_paragraph = 0

    text = ""
    for heading in document['meta']['heading'].values():
        if heading != None:
            text += f"{heading}\n"

    for doc in document['content']:
        if doc['category'] == 'paragraph':
            num_paragraph += 1
            tmp_text = preprocess_sentence(doc['text'])
            text += f" {tmp_text}"


    meta = {'num_letter' : len(text), 'num_paragraph': num_paragraph, 'document_index': document['meta']['index']}
    return text, meta


text_list = []
meta_list = []

for doc in state['documents']:
    text, meta = stuff_preprocess_document(doc)
    text_list.append(text)
    meta_list.append(meta)

stuff_dataset = (text, meta)


In [53]:
# map 전처리 

def map_preprocessing(document):
    text = [preprocess_sentence(content['text']) for content in document['content'] if content['category'] == 'paragraph']
    meta = {'num_letter' : len(''.join(text)), 'num_paragraph': len(text), 'document_index': document['meta']['index']}
    
    tmp = [element for element in document['meta']['heading'].values() if element != None]
    heading = " ".join(tmp if tmp != [] else '')
    
    return text, meta, heading

text_list, meta_list, heading_list = [], [], []

for doc in experiment_docs:
    text, meta, heading = map_preprocessing(doc)
    text_list.append(text)
    meta_list.append(meta)
    heading_list.append(heading)

map_dataset = (text_list, meta_list, heading_list)

### stuff

In [ ]:
stuff_prompt = """
# INSTRUCTIONS
Please summarize the sentence according to the following REQUEST.

REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Translate the summary into KOREAN if it is written in ENGLISH.
3. DO NOT translate any technical terms.
4. DO NOT include any unnecessary information.

# DOCUMENT:
{document}

# SUMMARY:"
"""

llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.0
)

stuff_prompt = PromptTemplate.from_template(stuff_prompt)
stuff_chain = stuff_prompt | llm
result = stuff_chain.invoke(text_list[0])

Ⅴ.역도 훈련프로그램 구성 및 지도안
1. 훈련프로그램의 구성 원리
 역도 훈련프로그램의 구성은 트레이닝의 원리에 입각하여 안전하고 효과적인 훈 련 프로그램을 설계하여 선수 개개인의 특수성, 과부하, 점진성의 일반적인 훈련원 칙에 근거한다.
content='- 역도 훈련프로그램은 트레이닝의 원리에 기반하여 구성됨.\n- 안전하고 효과적인 훈련 프로그램 설계.\n- 선수 개개인의 특수성, 과부하, 점진성에 따른 일반적인 훈련 원칙 적용.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 168, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None} id='run-645398fa-6b84-4cb9-b6eb-8e2487d35ea3-0' usage_metadata={'input_tokens': 168, 'output_tokens': 60, 'total_tokens': 228, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [53]:
print(result.content)
print(result.response_metadata['token_usage']['total_tokens'])

- 역도 훈련프로그램은 트레이닝의 원리에 기반하여 구성됨.
- 안전하고 효과적인 훈련 프로그램 설계.
- 선수 개개인의 특수성, 과부하, 점진성에 따른 일반적인 훈련 원칙 적용.
228


In [ ]:
def stuff_chain(chain, text, meta):
    tmp = chain.invoke(text)
    summary = tmp.content
    cost = tmp.response_metadata['token_usage']['total_tokens']

    return pd.DataFrame({'document_index': [meta['document_index']], 'num_paragraph': [meta['num_paragraph']], 'num_letter': [meta['num_letter']], 'summarization_method': ['stuff'], 'cost': [cost], 'response': [result.content], 'summary': [summary], 'clarity': [0], 'brevity': [0], 'completeness': [0], 'consistency': [0], 'usefulness': [0]})

In [ ]:
for text, meta in zip(text_list, meta_list):
    result = stuff_chain(stuff_chain, text, meta)
    result_df = pd.concat([result_df, result], ignore_index=True)

In [ ]:
result_df = pd.concat([result_df, result], ignore_index=True)

### map

In [9]:
map_prompt = """
# INSTRUCTIONS:
You are a professional main thesis extractor.
Your task is to extract main thesis from given documents. 
Answer should be written in {language}.

# DOCUMENTS:
{heading}
{documents}

# FORMAT: 
- thesis 1
- thesis 2
- ...

Write as many sentences as there are documents.
If there are more than 10 documents, write only 10 sentences.

# Answer:
"""

llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.0
)

map_prompt = PromptTemplate.from_template(map_prompt)
map_chain = map_prompt | llm

In [57]:
map_dataset[2][2]

'Ⅴ.역도 훈련프로그램 구성 및 지도안 1. 훈련프로그램의 구성 원리 나. 훈련량의 설정 및 훈련프로그램의 실례 1) 총부하량(volume of load)'

In [61]:
# 다음에 text_list로 되어 있는 부분을 고치는 것
# map의 경우, refine과 reduce 절차가 따라오니, 해당 결과를 진행한 후에 result_dataframe에 저장해야 함.


map_tmp_result = map_chain.invoke({'documents': map_dataset[0][2], 'language': 'KOREAN', 'heading': map_dataset[2][2]})

In [62]:
map_tmp_result_content = map_tmp_result.content

In [63]:
map_tmp_result_content

'- 총부하량은 특정 기간 동안 들어 올린 총중량을 의미하며, 킬로그램이나 톤으로 표시된다.\n- 훈련의 총부하량은 선수의 운동량을 나타내며, 주 단위, 월 단위, 년 단위로 산출된다.\n- 총부하량은 선수의 운동경력, 체급, 체력에 따라 달라져야 하며, 급격한 증가가 경기력에 부정적인 영향을 미칠 수 있다.\n- 과도한 운동량은 운동 상해를 초래하고 선수생활을 단축시킬 수 있다.\n- 훈련량의 증가뿐만 아니라 회복과 훈련 효과의 관계도 중요하다.\n- 선수는 과부하의 원리와 점진성의 원리를 적용하여 운동량을 설정해야 한다.\n- 월간 총부하량은 시합 1개월 전 개인 차에 따라 100~150톤에 이를 수 있으며, 일반 선수는 200~300톤의 부하량으로 훈련하는 경우가 있다.\n- 주간 총부하량은 시합 4주 전부터 시합 날까지 제한된 범위 내에서 설정해야 한다.\n- 부하량 감소 시 15~18 이상의 감소는 기록 유지 및 향상에 영향을 미친다.\n- 주간 총부하량은 훈련의 효과를 기대할 수 있는 기본 단위로, 훈련기와 이완기에 따라 달리 분배해야 한다.'

In [64]:
print(map_tmp_result_content)

- 총부하량은 특정 기간 동안 들어 올린 총중량을 의미하며, 킬로그램이나 톤으로 표시된다.
- 훈련의 총부하량은 선수의 운동량을 나타내며, 주 단위, 월 단위, 년 단위로 산출된다.
- 총부하량은 선수의 운동경력, 체급, 체력에 따라 달라져야 하며, 급격한 증가가 경기력에 부정적인 영향을 미칠 수 있다.
- 과도한 운동량은 운동 상해를 초래하고 선수생활을 단축시킬 수 있다.
- 훈련량의 증가뿐만 아니라 회복과 훈련 효과의 관계도 중요하다.
- 선수는 과부하의 원리와 점진성의 원리를 적용하여 운동량을 설정해야 한다.
- 월간 총부하량은 시합 1개월 전 개인 차에 따라 100~150톤에 이를 수 있으며, 일반 선수는 200~300톤의 부하량으로 훈련하는 경우가 있다.
- 주간 총부하량은 시합 4주 전부터 시합 날까지 제한된 범위 내에서 설정해야 한다.
- 부하량 감소 시 15~18 이상의 감소는 기록 유지 및 향상에 영향을 미친다.
- 주간 총부하량은 훈련의 효과를 기대할 수 있는 기본 단위로, 훈련기와 이완기에 따라 달리 분배해야 한다.


## reduce

In [58]:
reduce_prompt = """
# INSTRUCTIONS:
You are a professional summarizer. 
You are given a list of summaries of documents and you are asked to create a single summary of the documents.

1. Extract main points from a list of summaries of ducuments.
2. Make final summaries in bullet points format.
3. Answer should be written in {language}.

# CONTEXT:
{summaries}

# SUMMARY:
"""

In [59]:
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.0
)

reduce_prompt = PromptTemplate.from_template(reduce_prompt)
reduce_chain = reduce_prompt | llm

In [ ]:
# reduce_result = reduce_chain.invoke({'language': 'KOREAN', 'summaries': map_tmp_result_content})

In [17]:
reduce_result

AIMessage(content='- 총부하량은 특정 기간 동안 들어 올린 총중량을 킬로그램이나 톤으로 표시하며, 선수의 운동량을 나타낸다.\n- 훈련의 총부하량은 주 단위, 월 단위, 년 단위로 산출되며, 선수의 운동경력, 체급, 체력에 따라 달라져야 한다.\n- 급격한 부하량 증가는 경기력에 부정적인 영향을 미칠 수 있으며, 과도한 운동량은 운동 상해를 초래하고 선수생활을 단축시킬 수 있다.\n- 훈련량 증가와 회복의 관계가 중요하며, 초기 단계에서의 회복이 이루어지지 않으면 훈련 효과가 감소할 수 있다.\n- 적절한 부하량 설정은 최대의 경기력을 위한 필수적인 프로그램 구성에 반영되어야 한다.\n- 월간 총부하량은 시합 1개월 전에는 100-150톤에 이르며, 일반 선수들은 200-300톤의 과도한 부하량으로 훈련하는 경우가 많다.\n- 주간 총부하량은 시합 4주 전부터 시합 날까지 제한된 부하량 범위 내에서 설정해야 하며, 훈련의 효과를 기대할 수 있는 기본 단위로 다르게 분배해야 한다.\n- 부하량을 감소시킬 때에는 15-18 이상의 감소가 기록 유지 및 향상에 영향을 미친다는 점을 항상 염두에 두어야 한다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 334, 'prompt_tokens': 450, 'total_tokens': 784, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329

In [27]:
print(map_tmp_result_content)

- 총부하량은 특정 기간 동안 들어 올린 총중량을 의미하며, 킬로그램이나 톤으로 표시된다.
- 훈련의 총부하량은 선수의 운동량을 나타내며, 주 단위, 월 단위, 년 단위로 산출된다.
- 총부하량은 선수의 운동경력, 체급, 체력에 따라 달라져야 하며, 급격한 증가가 경기력에 부정적인 영향을 미칠 수 있다.
- 과도한 운동량은 운동 상해를 초래하고 선수생활을 단축시킬 수 있다.
- 훈련량의 증가뿐만 아니라 회복과의 관계도 중요하며, 초기 단계에서의 회복이 이루어지지 않으면 훈련 효과가 감소할 수 있다.
- 선수에게 적절한 부하량 설정은 최대의 경기력을 위한 필수적인 프로그램 구성에 반영되어야 한다.
- 월간 총부하량은 시합 1개월 전에는 100-150톤에 이르며, 일반 선수들은 200-300톤의 과도한 부하량으로 훈련하는 경우가 많다.
- 주간 총부하량은 시합 4주 전부터 시합 날까지 제한된 부하량 범위 내에서 설정해야 한다.
- 주간 총부하량은 훈련의 효과를 기대할 수 있는 기본 단위로, 시합기, 훈련기, 이완기에 따라 달리 분배해야 한다.
- 부하량을 감소시킬 때에는 15-18 이상의 감소가 기록 유지 및 향상에 영향을 미친다는 점을 항상 염두에 두어야 한다.


In [25]:
print(reduce_result.content)

- 총부하량은 특정 기간 동안 들어 올린 총중량을 킬로그램이나 톤으로 표시하며, 선수의 운동량을 나타낸다.
- 훈련의 총부하량은 주 단위, 월 단위, 년 단위로 산출되며, 선수의 운동경력, 체급, 체력에 따라 달라져야 한다.
- 급격한 부하량 증가는 경기력에 부정적인 영향을 미칠 수 있으며, 과도한 운동량은 운동 상해를 초래하고 선수생활을 단축시킬 수 있다.
- 훈련량 증가와 회복의 관계가 중요하며, 초기 단계에서의 회복이 이루어지지 않으면 훈련 효과가 감소할 수 있다.
- 적절한 부하량 설정은 최대의 경기력을 위한 필수적인 프로그램 구성에 반영되어야 한다.
- 월간 총부하량은 시합 1개월 전에는 100-150톤에 이르며, 일반 선수들은 200-300톤의 과도한 부하량으로 훈련하는 경우가 많다.
- 주간 총부하량은 시합 4주 전부터 시합 날까지 제한된 부하량 범위 내에서 설정해야 하며, 훈련의 효과를 기대할 수 있는 기본 단위로 다르게 분배해야 한다.
- 부하량을 감소시킬 때에는 15-18 이상의 감소가 기록 유지 및 향상에 영향을 미친다는 점을 항상 염두에 두어야 한다.


## refine

In [65]:
refine_prompt = """
# INSTRUCTIONS:
You are a expert summarizer.
Your hob is to produce a final summary.
We have provided an existing summary up to a certain point

We have the opportunity to refine the existing summary(only if needed) with some more context below.

# CURRENT SUMMARY:: 
{summaries}

Given the new context, refine the original summary in {language}.

# SUMMARY:
"""

In [66]:
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.0
)
refine_prompt = PromptTemplate.from_template(refine_prompt)
refine_chain = refine_prompt | llm

In [67]:
# refine_result = refine_chain.invoke({'summaries': map_tmp_result_content, 'language': 'KOREAN'})

In [75]:
print(refine_result.content)

- 총부하량은 특정 기간 동안 들어 올린 총중량을 의미하며, 킬로그램이나 톤으로 표시된다.
- 훈련의 총부하량은 선수의 운동량을 나타내며, 주 단위, 월 단위, 년 단위로 산출된다.
- 총부하량은 선수의 운동경력, 체급, 체력에 따라 달라져야 하며, 급격한 증가는 경기력에 부정적인 영향을 미칠 수 있다.
- 과도한 운동량은 운동 상해를 초래하고 선수생활을 단축시킬 수 있다.
- 훈련량의 증가뿐만 아니라 회복과 훈련 효과의 관계도 중요하다.
- 선수는 과부하의 원리와 점진성의 원리를 적용하여 운동량을 설정해야 한다.
- 월간 총부하량은 시합 1개월 전 개인 차에 따라 100~150톤에 이를 수 있으며, 일반 선수는 200~300톤의 부하량으로 훈련하는 경우가 있다.
- 주간 총부하량은 시합 4주 전부터 시합 날까지 제한된 범위 내에서 설정해야 하며, 훈련의 효과를 기대할 수 있는 기본 단위로, 훈련기와 이완기에 따라 달리 분배해야 한다.
- 부하량 감소 시 15~18% 이상의 감소는 기록 유지 및 향상에 영향을 미친다.


# 평가 과정

In [25]:
evaluation_prompt = """
# INSTRUCTIONS
You are tasked with evaluating the quality of a summary based on the original document. 
Please assess the summary according to the following criteria.

# ORIGINAL DOCUMENT:
{original_document}

# SUMMARY:
{summary}

# EVALUATION CRITERIA:
1. Clarity
   - Is the summary easy to understand?
   - Are there any ambiguous expressions?
   - Is the sentence structure logical?
   - Does the summary avoid complex jargon?
   - Is the information presented in a straightforward manner?

2. Conciseness
   - Does the summary avoid unnecessary repetition?
   - Is the information presented succinctly?
   - Are there any redundant details?
   - Does the summary focus on key points only?
   - Is the summary free of superfluous information?

3. Completeness
   - Does the summary include all major points from the original document?
   - Are any critical details missing?
   - Does the summary reflect the main ideas of the original document?
   - Is the summary comprehensive?
   - Are all essential elements of the original document covered?

4. Consistency
   - Does the summary align with the context of the original document?
   - Is the tone of the summary consistent with the original document?
   - Are there any contradictions in the summary?
   - Does the summary maintain a consistent style?
   - Is the summary logically coherent?

5. Usefulness
   - Does the summary provide valuable information?
   - Is the summary helpful for understanding the original document?
   - Does the summary serve a practical purpose?
   - Is the summary informative?
   - Can the summary stand alone as a useful piece of information?

# Format:
{format}
"""

In [26]:
"""
Please provide your evaluation for each criterion with 'yes' or 'no'.
{{
 'Clarity': [          ],
 'Conciseness':  [          ],
 'Completeness':  [          ],
 'Consistency': [          ],
 'Usefulness':  [          ]
}}
"""

"\nPlease provide your evaluation for each criterion with 'yes' or 'no'.\n{{\n 'Clarity': [          ],\n 'Conciseness':  [          ],\n 'Completeness':  [          ],\n 'Consistency': [          ],\n 'Usefulness':  [          ]\n}}\n"

In [27]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class CriterionResult(BaseModel):
    clarity: List[bool] = Field(descrioption="evaluations for each criterion of Clarity with 'yes' or 'no'")
    conciseness: List[bool] = Field(descrioption="evaluations for each criterion of Conciseness with 'yes' or 'no'")
    completeness: List[bool] = Field(descrioption="evaluations for each criterion of Completeness with 'yes' or 'no'")
    consistency: List[bool] = Field(descrioption="evaluations for each criterion of Consistency with 'yes' or 'no'")
    usefulness: List[bool] = Field(descrioption="evaluations for each criterion of Usefulness with 'yes' or 'no'")

parser = PydanticOutputParser(pydantic_object=CriterionResult)

In [28]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"clarity": {"descrioption": "evaluations for each criterion of Clarity with \'yes\' or \'no\'", "items": {"type": "boolean"}, "title": "Clarity", "type": "array"}, "conciseness": {"descrioption": "evaluations for each criterion of Conciseness with \'yes\' or \'no\'", "items": {"type": "boolean"}, "title": "Conciseness", "type": "array"}, "completeness": {"descrioption": "evaluations for each criterion of Completeness with \'yes\' or \'no\'", "items": {"type": "boolean"}, "title": "Completeness", "type": "array"}, "consi

In [29]:
evaluation_prompt = PromptTemplate.from_template(evaluation_prompt)
evaluation_prompt = evaluation_prompt.partial(format=parser.get_format_instructions())

In [30]:
llm = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0.0
)

evaluation_chain = evaluation_prompt | llm

In [54]:
evaluation = evaluation_chain.invoke({'original_document': tmp_origin, 'summary': tmp_summary})
structured_output = parser.parse(evaluation.content)

In [62]:
structured_output

CriterionResult(clarity=[True, False, True, True, True], conciseness=[True, True, True, True, True], completeness=[True, False, True, True, True], consistency=[True, True, True, True, True], usefulness=[True, True, True, True, True])

In [63]:
print(structured_output.clarity)
print(structured_output.conciseness)
print(structured_output.completeness)
print(structured_output.consistency)
print(structured_output.usefulness)

[True, False, True, True, True]
[True, True, True, True, True]
[True, False, True, True, True]
[True, True, True, True, True]
[True, True, True, True, True]


In [56]:
evaluation.response_metadata['token_usage']['total_tokens']

2383

# 잘못된 요약 생성

In [ ]:
지금은 이걸 하자.
이걸 한 후에, 실험을 위한 전체 로직을 수행하고, 결과를 반영하여, node를 만들자.
다음에 이미지, 표에 대한 전처리 만들자.
그걸로 전체 Document를 생성하자.
생성한 결과를 기반으로, ragas를 활용하여 hyper parameter tuning을 진행하자.
이렇게 하면, 되나??

스포츠 코칭의 intent인가 그거 분류하도록 chatbot 설계해야 하는데???
그거에 맞게 해야하는데.... 잘 모르겠네...

In [90]:
wrong_prompt = """
- Your task is to create a summary that fails to meet certain evaluation criteria, and to identify which criteria are not satisfied.
- This summary will be used to test another evaluation prompt that returns boolean (True/False) values for each criterion.

# Task 1: 
- From the evaluation criteria listed below, select the items that the summary does not meet (i.e., those marked as False).
- The number of selected items should be more than 2 for each criteria.
- The items selected for each evaluation criterion must be chosen randomly.
- ex) Clarity: [True, False, False, False, True] 


[EVALUATION CRITERIA]:
1. Clarity
   - Is the summary easy to understand?
   - Are there any ambiguous expressions?   
   - Is the sentence structure logical?
   - Does the summary avoid complex jargon?
   - Is the information presented in a straightforward manner?

2. Conciseness
   - Does the summary avoid unnecessary repetition?
   - Is the information presented succinctly?
   - Are there any redundant details?
   - Does the summary focus on key points only?
   - Is the summary free of superfluous information?

3. Completeness
   - Does the summary include all major points from the original document?
   - Are any critical details missing?
   - Does the summary reflect the main ideas of the original document?
   - Is the summary comprehensive?
   - Are all essential elements of the original document covered?

4. Consistency
   - Does the summary align with the context of the original document?
   - Is the tone of the summary consistent with the original document?
   - Are there any contradictions in the summary?
   - Does the summary maintain a consistent style?
   - Is the summary logically coherent?

5. Usefulness
   - Does the summary provide valuable information?
   - Is the summary helpful for understanding the original document?
   - Does the summary serve a practical purpose?
   - Is the summary informative?
   - Can the summary stand alone as a useful piece of information?
---

# Task 2: 
- summarize should be written in {language}.
- Based on the result of {{selected_criteria}}, summarize the following original text:

[ORIGIN TEXT]:
{original_text}

---
# Format:
   {format}
"""

In [91]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class CriteriaAndSummary(BaseModel):
    clarity: List[bool] = Field(description="result of clarity in {{selected_criteria}}")
    conciseness: List[bool] = Field(description="result of conciseness in {{selected_criteria}}'s")
    completeness: List[bool] = Field(description="result of ompleteness in {{selected_criteria}}'s")
    consistency: List[bool] = Field(description="result of consistency in {{selected_criteria}}'s")
    usefulness: List[bool] = Field(description="result of usefulness in {{selected_criteria}}'s")
    summary: str = Field(description="summary")

In [92]:
parser = PydanticOutputParser(pydantic_object=CriteriaAndSummary)
print(parser.get_format_instructions())
wrong_prompt = PromptTemplate.from_template(wrong_prompt)
wrong_prompt = wrong_prompt.partial(format=parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"clarity": {"description": "result of clarity in {{selected_criteria}}", "items": {"type": "boolean"}, "title": "Clarity", "type": "array"}, "conciseness": {"description": "result of conciseness in {{selected_criteria}}'s", "items": {"type": "boolean"}, "title": "Conciseness", "type": "array"}, "completeness": {"description": "result of ompleteness in {{selected_criteria}}'s", "items": {"type": "boolean"}, "title": "Completeness", "type": "array"}, "consistency": {"description": "result of consistency in {{selected_criteria}}'s

In [93]:
chain = wrong_prompt | llm
result = chain.invoke({'original_text': tmp, 'language': 'KOREAN'})

In [96]:
parsed = parser.parse(result.content)

In [101]:
parsed.clarity[0]




True